In [ ]:
pip install opencv-python numpy csv

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [ ]:
!pip install opencv-contrib-python

In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import numpy as np
import os
import csv
import mediapipe as mp
from datetime import datetime
import pytz
from base64 import b64decode
from google.colab.output import eval_js
from IPython.display import display, Javascript

# Initialize FaceMesh for Liveness Detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Webcam Photo Capture (Google Colab)
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js(f'takePhoto({quality})')
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


class LivenessDetector:
    def __init__(self):
        self.ear_history = []
        self.nose_positions = []
        self.blink_count = 0
        self.movement_threshold = 10  # Adjusted for better detection
        self.ear_threshold = 0.21
        self.avg_movement = 0

    def calculate_ear(self, eye_landmarks):
        d1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
        d2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
        d3 = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
        return (d1 + d2) / (2 * d3)

    def detect(self, image):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image_rgb)

        if not results.multi_face_landmarks:
            return False

        landmarks = results.multi_face_landmarks[0].landmark
        h, w, _ = image.shape

        left_eye = np.array([(landmarks[i].x * w, landmarks[i].y * h)
                           for i in [33, 160, 158, 133, 153, 144]])
        right_eye = np.array([(landmarks[i].x * w, landmarks[i].y * h)
                            for i in [362, 385, 387, 263, 373, 380]])
        nose = (landmarks[4].x * w, landmarks[4].y * h)

        left_ear = self.calculate_ear(left_eye)
        right_ear = self.calculate_ear(right_eye)
        avg_ear = (left_ear + right_ear) / 2

        self.ear_history.append(avg_ear)
        if len(self.ear_history) > 5:
            self.ear_history.pop(0)

        if len(self.ear_history) == 5:
            if (self.ear_history[0] > self.ear_threshold and
                min(self.ear_history[1:4]) < self.ear_threshold and
                self.ear_history[4] > self.ear_threshold):
                self.blink_count += 1
                print(f"Blink detected! Total blinks: {self.blink_count}")

        self.nose_positions.append(nose)
        if len(self.nose_positions) > 10:
            self.nose_positions.pop(0)

        if len(self.nose_positions) >= 2:
            movements = [np.linalg.norm(np.array(a)-np.array(b))
                        for a,b in zip(self.nose_positions[1:], self.nose_positions[:-1])]
            self.avg_movement = np.mean(movements)
        else:
            self.avg_movement = 0

        return self.blink_count >= 2 and self.avg_movement > self.movement_threshold

class FaceAttendanceSystem:
    def __init__(self):
        self.face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        self.face_recognizer = cv2.face.LBPHFaceRecognizer_create()
        self.dataset_dir = "dataset"
        self.registered_names = []
        os.makedirs(self.dataset_dir, exist_ok=True)

        if os.path.exists("registered_names.npy"):
            self.registered_names = np.load("registered_names.npy", allow_pickle=True).tolist()

    def register_face(self):
        name = input("Enter your name: ").strip()
        if name in self.registered_names:
            print("User already exists!")
            return

        person_dir = os.path.join(self.dataset_dir, name)
        os.makedirs(person_dir, exist_ok=True)

        print("Capturing images... Look at the camera and blink/move your head.")
        for i in range(10):
            filename = os.path.join(person_dir, f"{i}.jpg")
            take_photo(filename)
            print(f"Captured {i+1}/10")

        self.registered_names.append(name)
        np.save("registered_names.npy", self.registered_names)
        print("✅ Face registered successfully!")

        print("🔄 Training model...")
        self.train_model()
        print("✅ Model trained successfully!")

    def train_model(self):
        faces, labels = [], []
        label_map = {name: idx for idx, name in enumerate(self.registered_names)}

        for name, label in label_map.items():
            person_dir = os.path.join(self.dataset_dir, name)
            for filename in os.listdir(person_dir):
                img_path = os.path.join(person_dir, filename)
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                faces.append(image)
                labels.append(label)

        self.face_recognizer.train(faces, np.array(labels))
        self.face_recognizer.save("trained_model.yml")

    def log_attendance(self, name, action):
        tz = pytz.timezone('Asia/Karachi')
        now = datetime.now(tz)
        date_str, time_str = now.strftime("%Y-%m-%d"), now.strftime("%H:%M:%S")
        csv_path = os.path.join(os.getcwd(), "attendance.csv")

        file_exists = os.path.isfile(csv_path)
        with open(csv_path, "a", newline='') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(["Name", "Date", "Time", "Action"])
            writer.writerow([name, date_str, time_str, action])
        print(f"✅ {name} marked {action} at {date_str} {time_str}")

    def recognize_face(self, action_type):
        if not os.path.exists("trained_model.yml"):
            print("❌ No trained model found! Register a user first.")
            return

        self.face_recognizer.read("trained_model.yml")

        print(f"🔍 Performing liveness check for {action_type}...")
        while True:
            filename = take_photo()
            image = cv2.imread(filename)
            if image is None:
                continue

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = self.face_detector.detectMultiScale(gray, 1.1, 5)

            for (x, y, w, h) in faces:
                label, conf = self.face_recognizer.predict(gray[y:y+h, x:x+w])
                name = self.registered_names[label] if conf < 100 else "Unknown"

                if name != "Unknown":
                    self.log_attendance(name, action_type)
                    print(f"✅ {name} logged {action_type} successfully!")
                    os.remove(filename)
                    return
            os.remove(filename)
            if input("Press 'q' to quit: ").lower() == 'q':
                break

# Main Program
if __name__ == "__main__":
    system = FaceAttendanceSystem()

    while True:
        print("\n1. Register Face\n2. Log In\n3. Log Out\n4. Exit")
        choice = input("Choose option: ").strip()

        if choice == '1':
            system.register_face()
        elif choice == '2':
            system.recognize_face("in")
        elif choice == '3':
            system.recognize_face("out")
        elif choice == '4':
            break
        else:
            print("❌ Invalid choice! Please try again.")



1. Register Face
2. Log In
3. Log Out
4. Exit
Choose option: 2
🔍 Performing liveness check for in...


<IPython.core.display.Javascript object>

✅ Hassan marked in at 2025-02-19 12:07:32
✅ Hassan logged in successfully!

1. Register Face
2. Log In
3. Log Out
4. Exit
Choose option: 3
🔍 Performing liveness check for out...


<IPython.core.display.Javascript object>

✅ Hassan marked out at 2025-02-19 12:07:42
✅ Hassan logged out successfully!

1. Register Face
2. Log In
3. Log Out
4. Exit
Choose option: 4
